In [12]:
import torch
import zipfile
import numpy as np
import sys, random
import pandas as pd
import torch.nn as nn
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
from torchvision import models, transforms

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def extract_zip_file(zip_path, dest_path):
    """
    This function extracts a zip file to a specified destination path.

    Parameters:
    zip_path (str): The path to the zip file that needs to be extracted.
    dest_path (str): The path where the zip file should be extracted to.

    Returns:
    None
    """
    # Open the zip file in read mode
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract all the contents of the zip file to dest_path
        zip_ref.extractall(dest_path)

# Usage
# The function is called with the path of the zip file and the destination path as arguments
extract_zip_file('/content/drive/MyDrive/data.zip', 'data')

In [9]:
# Path for model
MODEL = '/content/drive/MyDrive/model_finetune.pth'
CSV_FILE = "/content/data/X_test.csv"
OUTPUT_FILE = "/content/data/submission_finetune.csv"

# Load the model for testing
model = torch.load(MODEL)
model.eval()

# Class labels for prediction
class_names = ['0', '1']

# Preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize(size=256),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Enable gpu mode, if cuda available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the CSV file
df = pd.read_csv(CSV_FILE)

# Create a new DataFrame for the output
output_df = pd.DataFrame(columns=['output'])

# Perform prediction for each image in the CSV file
with torch.no_grad():
    for index, row in df.iterrows():
        img_path = Path(row['image_path']).resolve()
        img = Image.open(img_path).convert('RGB')
        inputs = preprocess(img).unsqueeze(0).to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        output_df.loc[index, 'output'] = class_names[preds.item()]

# Save the output DataFrame to a new CSV file
output_df.to_csv(OUTPUT_FILE, index=False)

In [13]:
def vgg_block_single(in_ch, out_ch, kernel_size=3, padding=1):
    """
    This function defines a single convolutional block for VGG network.

    Parameters:
    in_ch (int): The number of input channels.
    out_ch (int): The number of output channels.
    kernel_size (int): The size of the kernel. Default is 3.
    padding (int): The size of the padding. Default is 1.

    Returns:
    nn.Sequential: A sequential container of Conv2d, ReLU, and MaxPool2d.
    """
    return nn.Sequential(
        nn.Conv2d(in_ch, out_ch, kernel_size=kernel_size, padding=padding),  # Convolutional layer
        nn.ReLU(),  # Activation function
        nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling layer
    )

def vgg_block_double(in_ch, out_ch, kernel_size=3, padding=1):
    """
    This function defines a double convolutional block for VGG network.

    Parameters:
    in_ch (int): The number of input channels.
    out_ch (int): The number of output channels.
    kernel_size (int): The size of the kernel. Default is 3.
    padding (int): The size of the padding. Default is 1.

    Returns:
    nn.Sequential: A sequential container of two Conv2d, two ReLU, and one MaxPool2d.
    """
    return nn.Sequential(
        nn.Conv2d(in_ch, out_ch, kernel_size=kernel_size, padding=padding),  # First convolutional layer
        nn.ReLU(),  # First activation function
        nn.Conv2d(out_ch, out_ch, kernel_size=kernel_size, padding=padding),  # Second convolutional layer
        nn.ReLU(),  # Second activation function
        nn.MaxPool2d(kernel_size=2, stride=2)  # Max pooling layer
    )

class MyVGG11(nn.Module):
    """
    This class defines the VGG11 architecture.

    Attributes:
    conv_block1 (nn.Sequential): The first convolutional block.
    conv_block2 (nn.Sequential): The second convolutional block.
    conv_block3 (nn.Sequential): The third convolutional block.
    conv_block4 (nn.Sequential): The fourth convolutional block.
    conv_block5 (nn.Sequential): The fifth convolutional block.
    fc_layers (nn.Sequential): The fully connected layers.
    """
    def __init__(self, in_ch, num_classes):
        super().__init__()

        # Define the convolutional blocks
        self.conv_block1 = vgg_block_single(in_ch, 64)
        self.conv_block2 = vgg_block_single(64, 128)
        self.conv_block3 = vgg_block_double(128, 256)
        self.conv_block4 = vgg_block_double(256, 512)
        self.conv_block5 = vgg_block_double(512, 512)

        # Define the fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),  # First fully connected layer
            nn.ReLU(inplace=True),  # Activation function
            nn.Dropout(),  # Dropout layer
            nn.Linear(4096, 4096),  # Second fully connected layer
            nn.ReLU(inplace=True),  # Activation function
            nn.Dropout(),  # Dropout layer
            nn.Linear(4096, num_classes)  # Output layer
        )

    def forward(self, x):
        """
        This function defines the forward pass of the VGG11 network.

        Parameters:
        x (torch.Tensor): The input tensor.

        Returns:
        torch.Tensor: The output tensor.
        """
        # Pass the input through the convolutional blocks
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)

        # Flatten the tensor
        x = x.view(x.size(0), -1)

        # Pass the flattened tensor through the fully connected layers
        x = self.fc_layers(x)

        return x


In [14]:
# Define the path for the model, input CSV file, and output CSV file
MODEL = '/content/drive/MyDrive/model_scratch.pth'
CSV_FILE = "/content/data/X_test.csv"
OUTPUT_FILE = "/content/data/submission_scratch.csv"

# Load the model for testing
model = torch.load(MODEL)
model.eval()  # Set the model to evaluation mode

# Define the class labels for prediction
class_names = ['0', '1']

# Define the preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize(size=256),  # Resize the image
    transforms.CenterCrop(size=224),  # Crop the image from the center
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the image
])

# Enable GPU mode, if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the CSV file
df = pd.read_csv(CSV_FILE)

# Create a new DataFrame for the output
output_df = pd.DataFrame(columns=['output'])

# Perform prediction for each image in the CSV file
with torch.no_grad():  # Disable gradient calculation
    for index, row in df.iterrows():
        img_path = Path(row['image_path']).resolve()  # Get the image path
        img = Image.open(img_path).convert('RGB')  # Open the image and convert it to RGB
        inputs = preprocess(img).unsqueeze(0).to(device)  # Preprocess the image and move it to the device
        outputs = model(inputs)  # Get the model's predictions
        _, preds = torch.max(outputs, 1)  # Get the predicted class
        output_df.loc[index, 'output'] = class_names[preds.item()]  # Add the prediction to the output DataFrame

# Save the output DataFrame to a new CSV file
output_df.to_csv(OUTPUT_FILE, index=False)


In [ ]:
# Define the path for the model, input CSV file, and output CSV file
MODEL = '/content/drive/MyDrive/model_transfer.pth'
CSV_FILE = "/content/data/X_test.csv"
OUTPUT_FILE = "/content/data/submission_transfer.csv"

# Load the model for testing
model = torch.load(MODEL)
model.eval()  # Set the model to evaluation mode

# Define the class labels for prediction
class_names = ['0', '1']

# Define the preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize(size=256),  # Resize the image
    transforms.CenterCrop(size=224),  # Crop the image from the center
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the image
])

# Enable GPU mode, if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the CSV file
df = pd.read_csv(CSV_FILE)

# Create a new DataFrame for the output
output_df = pd.DataFrame(columns=['output'])

# Perform prediction for each image in the CSV file
with torch.no_grad():  # Disable gradient calculation
    for index, row in df.iterrows():
        img_path = Path(row['image_path']).resolve()  # Get the image path
        img = Image.open(img_path).convert('RGB')  # Open the image and convert it to RGB
        inputs = preprocess(img).unsqueeze(0).to(device)  # Preprocess the image and move it to the device
        outputs = model(inputs)  # Get the model's predictions
        _, preds = torch.max(outputs, 1)  # Get the predicted class
        output_df.loc[index, 'output'] = class_names[preds.item()]  # Add the prediction to the output DataFrame

# Save the output DataFrame to a new CSV file
output_df.to_csv(OUTPUT_FILE, index=False)


In [ ]:
# Define the path for the model, input CSV file, and output CSV file
MODEL = '/content/drive/MyDrive/model_finetune.pth'
CSV_FILE = "/content/data/X_test.csv"
OUTPUT_FILE = "/content/data/submission_finetune.csv"

# Load the model for testing
model = torch.load(MODEL)
model.eval()  # Set the model to evaluation mode

# Define the class labels for prediction
class_names = ['0', '1']

# Define the preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize(size=256),  # Resize the image
    transforms.CenterCrop(size=224),  # Crop the image from the center
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the image
])

# Enable GPU mode, if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the CSV file
df = pd.read_csv(CSV_FILE)

# Create a new DataFrame for the output
output_df = pd.DataFrame(columns=['output'])

# Perform prediction for each image in the CSV file
with torch.no_grad():  # Disable gradient calculation
    for index, row in df.iterrows():
        img_path = Path(row['image_path']).resolve()  # Get the image path
        img = Image.open(img_path).convert('RGB')  # Open the image and convert it to RGB
        inputs = preprocess(img).unsqueeze(0).to(device)  # Preprocess the image and move it to the device
        outputs = model(inputs)  # Get the model's predictions
        _, preds = torch.max(outputs, 1)  # Get the predicted class
        output_df.loc[index, 'output'] = class_names[preds.item()]  # Add the prediction to the output DataFrame

# Save the output DataFrame to a new CSV file
output_df.to_csv(OUTPUT_FILE, index=False)
